In [1]:
from Bio.PDB import PDBParser
import pandas as pd
import numpy as np
import os
import rdkit.Chem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from tqdm import tqdm
import glob
import torch
import torch.nn.functional as F
from io import StringIO
import sys
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.PDBIO import Select
import scipy
import scipy.spatial
import requests
from rdkit.Geometry import Point3D

def read_pdbbind_data(fileName):
    with open(fileName) as f:
        a = f.readlines()
    info = []
    for line in a:
        if line[0] == '#':
            continue
        lines, ligand = line.split('//')
        pdb, resolution, year, affinity, raw = lines.strip().split('  ')
        ligand = ligand.strip().split('(')[1].split(')')[0]
        # print(lines, ligand)
        info.append([pdb, resolution, year, affinity, raw, ligand])
    info = pd.DataFrame(info, columns=['pdb', 'resolution', 'year', 'affinity', 'raw', 'ligand'])
    info.year = info.year.astype(int)
    info.affinity = info.affinity.astype(float)
    return info


def read_mol(sdf_fileName, mol2_fileName, verbose=False):
    Chem.WrapLogs()
    stderr = sys.stderr
    sio = sys.stderr = StringIO()
    mol = Chem.MolFromMolFile(sdf_fileName, sanitize=False)   #先尝试读取.sdf文件
    problem = False
    try:
        Chem.SanitizeMol(mol)
        mol = Chem.RemoveHs(mol)
        sm = Chem.MolToSmiles(mol)
    except Exception as e:
        sm = str(e)
        problem = True
    if problem:                #如果.sdf文件无法读取，尝试读取.mol2文件
        mol = Chem.MolFromMol2File(mol2_fileName, sanitize=False)
        problem = False
        try:
            Chem.SanitizeMol(mol)
            mol = Chem.RemoveHs(mol)
            sm = Chem.MolToSmiles(mol)
            problem = False
        except Exception as e:     #如果两个文件都读不了，就返回错误提示
            sm = str(e)
            problem = True

    if verbose:
        print(sio.getvalue())
    sys.stderr = stderr
    return mol, problem


def write_sdf(toFile, sdf_fileName, mol2_fileName):
    # read in mol
    mol, _ = read_mol(sdf_fileName, mol2_fileName)
    
    w = Chem.SDWriter(toFile)      #用于将分子对象（mol）写入SDF文件
    w.write(mol)
    w.close()


def write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName):
    # read in mol
    mol, _ = read_mol(sdf_fileName, mol2_fileName)
    # reorder the mol atom number as in smiles.
    m_order = list(mol.GetPropsAsDict(includePrivate=True, includeComputed=True)['_smilesAtomOutputOrder'])  #用于获取分子对象（mol）的所有属性和值，并以字典的形式返回
    # TODO
    mol = Chem.RenumberAtoms(mol, m_order)      #用来对分子中的原子进行重新编号，以便与另一个分子或模板的原子顺序一致
    w = Chem.SDWriter(toFile)      #用于将分子对象（mol）写入SDF文件
    w.write(mol)
    w.close()



### Here : 筛去rdkit无法读取的配体分子

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
#from utils import read_pdbbind_data
# raw PDBbind dataset could be downloaded from http://www.pdbbind.org.cn/download.php
#pre = "./pdbbind_0.2.0/pdbbind2020"
df_pdb_id = pd.read_csv(f'./pdbbind_index_copy/INDEX_general_PL_name.2020', sep="  ", comment='#', header=None, names=['pdb', 'year', 'uid', 'd', 'e','f','g','h','i','j','k','l','m','n','o'], engine='python')
df_pdb_id = df_pdb_id[['pdb','uid']]
data = read_pdbbind_data(f'./pdbbind_index_copy/INDEX_general_PL_data.2020')
data = data.merge(df_pdb_id, on=['pdb'])


In [4]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
pdb_list = []
probem_list = []
num = 1
for pdb in tqdm(data.pdb):
    #print(num)
    num+=1
    sdf_fileName = f"./pdbbind_files_copy/{pdb}/{pdb}_ligand.sdf"
    mol2_fileName = f"./pdbbind_files_copy/{pdb}/{pdb}_ligand.mol2"
    mol, problem = read_mol(sdf_fileName, mol2_fileName)
    if problem:
        probem_list.append(pdb)
        continue
    pdb_list.append(pdb)

100%|██████████| 19443/19443 [00:24<00:00, 797.44it/s]


In [5]:
data = data.query("pdb in @pdb_list").reset_index(drop=True)

In [6]:
data.shape

(19119, 7)

In [7]:
print(len(pdb_list))
print(pdb_list[0])

19119
3zzf


### Here : 将能被rdkit读取的文件另外保存

In [ ]:
# 第一次运行后可不再运行
count = 0
for proteinName in tqdm(pdb_list):
    count += 1
    sdf_fileName = f"./pdbbind_files_copy/{proteinName}/{proteinName}_ligand.sdf"
    mol2_fileName = f"./pdbbind_files_copy/{proteinName}/{proteinName}_ligand.mol2"
    toFile = f"./rdkit_able_pdbbind_files_copy/{proteinName}.sdf"
    write_sdf(toFile, sdf_fileName, mol2_fileName)
print(count)

### Here : 为读取的配体分子中的原子重新编号，使其与simile表达式中的原子对齐

In [ ]:
# 第一次运行后可不再运行
count = 0
for proteinName in tqdm(pdb_list):
    count += 1
    sdf_fileName = f"./pdbbind_files_copy/{proteinName}/{proteinName}_ligand.sdf"
    mol2_fileName = f"./pdbbind_files_copy/{proteinName}/{proteinName}_ligand.mol2"
    toFile = f"./renumber_pdbbind_files_copy/{proteinName}_renumber.sdf"
    write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName)
print(count)

### Here : 对分子进行切分

In [8]:
tankbind_src_folder_path = "./util/"
import sys
sys.path.insert(0, tankbind_src_folder_path)

In [9]:
from segment_ligand import *
from tqdm import tqdm

ligand_segment_barycenter = {}
ligand_segment_inch = {}

for pdb in tqdm(pdb_list):
    ligand_file = f"./renumber_pdbbind_files_copy/{pdb}_renumber.sdf"

    barycenter, fragment_inch = segment_ligand(ligand_file)

    ligand_segment_barycenter[pdb] = barycenter

    ligand_segment_inch[pdb] = fragment_inch




100%|██████████| 19119/19119 [05:47<00:00, 54.95it/s]


In [10]:
print(ligand_segment_barycenter['1a0q'])
print(ligand_segment_inch['1a0q'])

{0: [13.317899069238848, 20.686142728698236, 58.386831491951156], 1: [12.483, 21.182, 60.518], 2: [11.888208598411099, 19.920876500661183, 63.35210247706711], 3: [15.97316666666667, 21.566333333333333, 55.06100000000001]}
{0: 'CKVICYBZYGZLLP-UHFFFAOYSA-N', 1: 'QGZKDVFQNNGYKY-UHFFFAOYSA-N', 2: 'UIUJIQZEACWQSV-UHFFFAOYSA-N', 3: 'UHOVQNZJYSORNB-UHFFFAOYSA-N'}


In [11]:
# 1 {0: [41.001250000000006, -6.6755, 48.17475], 1: [42.05448178553103, -9.73794088736464, 45.88512382868407], 2: [43.99406181283087, -7.498015899751973, 43.29957487135824], 3: [42.346, -7.452, 41.662], 4: [41.95795714175891, -8.174113910670357, 40.02429095556581], 5: [39.61183333333334, -5.689500000000001, 37.077], 6: [39.51566169316626, -3.967760113720122, 33.6366873960198], 7: [42.169, -8.311, 38.208], 8: [43.67024474650808, -8.016124834461907, 37.27960105944379]}

# 2 {0: [41.001250000000006, -6.6755, 48.17475], 1: [42.05448178553103, -9.73794088736464, 45.88512382868407], 2: [43.99406181283087, -7.498015899751973, 43.29957487135824], 3: [42.346, -7.452, 41.662], 4: [41.95795714175891, -8.174113910670357, 40.02429095556581], 5: [39.61183333333334, -5.689500000000001, 37.077], 6: [39.51566169316626, -3.967760113720122, 33.6366873960198], 7: [42.169, -8.311, 38.208], 8: [43.67024474650808, -8.016124834461907, 37.27960105944379]}


# {0: [13.317899069238848, 20.686142728698236, 58.386831491951156], 1: [12.483, 21.182, 60.518], 2: [11.888208598411099, 19.920876500661183, 63.35210247706711], 3: [15.97316666666667, 21.566333333333333, 55.06100000000001]}
# {0: 'CKVICYBZYGZLLP-UHFFFAOYSA-N', 1: 'QGZKDVFQNNGYKY-UHFFFAOYSA-N', 2: 'UIUJIQZEACWQSV-UHFFFAOYSA-N', 3: 'UHOVQNZJYSORNB-UHFFFAOYSA-N'}

In [12]:
from sortedcontainers import SortedSet

inchSet = SortedSet()

for i in ligand_segment_inch.keys():
    for j in ligand_segment_inch[i].keys():
        inchSet.add(ligand_segment_inch[i][j])

print(len(inchSet))

6770


In [13]:
for i in inchSet:
    if len(i)!=27:
        print(i)

CCCCCCCCCCCC(=O)[N+]12c3ccccn3~[Rh+2]1~n1ccccc12
CC[C@H](CCCCB(~[OH-])([OH2+])[OH2+])C(=O)[O-]
CC[C@]([NH3+])(CCCCB(O)(O)~[OH-])C(=O)[O-]
CC[C@]([NH3+])(CCCCB(~[OH-])([OH2+])[OH2+])C(=O)[O-]
C[C@]([NH3+])(CCC[CH2]~B(O)(O)O)C(=O)[O-]
[NH3+][C@@H](CCCCB(O)(O)~[OH-])C(=O)[O-]
[NH3+][C@@](CCCO)(CCCCB(~[OH-])([OH2+])[OH2+])C(=O)[O-]
[NH3+][C@@](CCC[CH2]~B(O)(O)O)(C(=O)[O-])C(F)F
[NH3+][C@H](CCC[CH2]~B(O)(O)O)C(=O)[O-]


### Here : 构建Subpocket和Index之间的映射

In [14]:
subpocket = []

for molecular in ligand_segment_inch.keys():
    for i in ligand_segment_inch[molecular].keys():
        subpocket_name = f"{molecular}-{i}"
        subpocket.append(subpocket_name)


In [15]:
subpocket.sort()

print(len(subpocket))
#for i in subpocket:
#    print(i)

132363


In [16]:
Subpocket_to_Index = {}
Index_to_Subpocket = {}

for i in range(len(subpocket)):
    Subpocket_to_Index[subpocket[i]] = i
    Index_to_Subpocket[i] = subpocket[i]


In [17]:
import json

with open('./Subpocket_to_Index.json', 'w') as f:
    json.dump(Subpocket_to_Index, f)

with open('./Index_to_Subpocket.json', 'w') as f:
    json.dump(Index_to_Subpocket, f)

### Here : 构建Motif和subpocket之间的映射

In [18]:
Motif_to_Subpocket = {}

for molecular in ligand_segment_inch.keys():
    for i in ligand_segment_inch[molecular].keys():

        motif_name = ligand_segment_inch[molecular][i]   # 主要疑问是这两个编号是否按照同一顺序！！！
        subpocket_name = f"{molecular}-{i}"
        
        if motif_name not in Motif_to_Subpocket.keys():
            Motif_to_Subpocket[motif_name] = []
            Motif_to_Subpocket[motif_name].append(subpocket_name)
        else:
            Motif_to_Subpocket[motif_name].append(subpocket_name)



In [19]:
print(len(Motif_to_Subpocket))
for i in Motif_to_Subpocket.keys():
    print(i,Motif_to_Subpocket[i])

6770
IKHGUXGNUITLKF-UHFFFAOYSA-N ['3zzf-0', '1w8l-0', '1zsb-0', '6k2n-0', '6k2n-7', '3e81-0', '4bh4-0', '4bh4-11', '5ele-0', '5ele-10', '4d63-0', '1hgj-4', '4bgx-0', '5ggl-0', '1rdn-0', '1mwt-1', '1mwt-3', '3ehn-0', '4bh3-0', '4bh3-11', '5nn4-0', '1ing-0', '4awj-3', '2ks9-19', '2zg3-0', '2zg1-0', '4aml-7', '3gv9-0', '1uxb-0', '3gqz-0', '6g47-0', '6g47-8', '4x11-0', '5hcl-0', '5v6u-0', '1hge-4', '1inh-0', '1uxa-0', '2hdu-0', '5enc-0', '5enj-0', '5elf-0', '1f73-0', '1nwl-0', '3t2t-7', '4u5s-0', '4d62-0', '1gbq-23', '4u5n-2', '5n53-3', '4wef-0', '5tpb-0', '2kfh-9', '5eld-0', '5eld-10', '6prt-1', '1hgi-4', '1hgi-6', '2ha4-0', '2krd-27', '3rqw-0', '4yrr-0', '2qwb-0', '5oyd-0', '3fcq-0', '6er4-0', '4yw2-0', '5fov-0', '4yz5-0', '4x47-0', '6jjm-0', '6jjm-13', '2woq-0', '4gqq-2', '6mvu-0', '1ms0-0', '4lm3-0', '4x14-0', '4x14-16', '6fv4-0', '2vmc-0', '4bgy-0', '4bgy-11', '4k3l-0', '2hw2-3', '2rnw-20', '2rnw-26', '2rnw-28', '2r0y-0', '3ap7-0', '4k0o-7', '4k64-0', '4lkh-0', '4u0w-0', '5c7b-1', '5t

In [20]:
with open('./Motif_to_Subpocket.json', 'w') as f:
    json.dump(Motif_to_Subpocket, f)

In [21]:

with open('./generate_HugeGraph_test2/Motif_to_Index.json', 'r') as f:
    Motif_to_Index = json.load(f)

In [22]:
MotifIndex_to_SubpocketIndex = {}

for i in Motif_to_Subpocket.keys():
    MotifIndex = Motif_to_Index[i]

    MotifIndex_to_SubpocketIndex[MotifIndex] = []

    for j in Motif_to_Subpocket[i]:
        SubpocketIndex = Subpocket_to_Index[j]

        MotifIndex_to_SubpocketIndex[MotifIndex].append(SubpocketIndex)
    


In [23]:
print(len(MotifIndex_to_SubpocketIndex))
for i in MotifIndex_to_SubpocketIndex.keys():
    print(i,MotifIndex_to_SubpocketIndex[i])

6770
2071 [59932, 15654, 17671, 126078, 126087, 40251, 62132, 62135, 93085, 93087, 63785, 5301, 62098, 95193, 12723, 9118, 9120, 40453, 62119, 62122, 103890, 6314, 61064, 24405, 35789, 35784, 60649, 43003, 14887, 42918, 122499, 122508, 84697, 95980, 109345, 5287, 6320, 14879, 22309, 93229, 93260, 93101, 3441, 9847, 54962, 82537, 63780, 4395, 82524, 103055, 83986, 107440, 24198, 93069, 93071, 129601, 5293, 5295, 22244, 24377, 53263, 86871, 29639, 105678, 41514, 120161, 87026, 94757, 87185, 84959, 125877, 125882, 32783, 67581, 127108, 8980, 73339, 84721, 84729, 122113, 31442, 62106, 62109, 71753, 22577, 30396, 30402, 30404, 29796, 36448, 71658, 71881, 73288, 82445, 90428, 107445, 107453, 107456, 123143, 3444, 4656, 88798, 24430, 63570, 63575, 226, 48233, 48243, 79061, 95205, 95215, 101640, 101651, 7482, 60562, 24455, 689, 34920, 7590, 84678, 84686, 114393, 94737, 94779, 1059, 12947, 67960, 91404, 3211, 102948, 120153, 54928, 71889, 66053, 66057, 66048, 7598, 70475, 84702, 84711, 102111, 

In [24]:
with open('./MotifIndex_to_SubpocketIndex-final.json', 'w') as f:
    json.dump(MotifIndex_to_SubpocketIndex, f)

### Here : 按切分块中心坐标切分蛋白质分子

In [25]:
"""
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBIO, PDBParser, Select

class DistanceSelect(Select):   # 定义一个名为DistanceSelect的类，继承自Bio.PDB的Select类。初始化时接收一个参考点和截止距离。
    def __init__(self, reference_point, cutoff):
        self.reference_point = reference_point
        self.cutoff = cutoff

    def accept_atom(self, atom):    # 在DistanceSelect类中定义一个名为accept_atom的方法。该方法计算输入原子与参考点的距离，如果距离小于等于截止距离，返回True，表示该原子被选中。
        distance = np.linalg.norm(self.reference_point - np.array(atom.coord))
        return distance <= self.cutoff


for protein_name in tqdm(ligand_segment_barycenter.keys()):
    toFolder = f"./cut_off_protein_pocket/{protein_name}"
    os.system(f"mkdir -p {toFolder}")

    barycenter = ligand_segment_barycenter[protein_name]
    for i in barycenter:
        # 读取PDB文件
        #pdb = "1a0q"
        pdb_file = f"./pdbbind_files_copy/{protein_name}/{protein_name}_protein.pdb"

        # 输入一个点的坐标
        #reference_point = np.array([15.97316666666667, 21.566333333333333, 55.06100000000001])
        reference_point = np.array(barycenter[i])

        # 设置截止距离
        cutoff = 10.0  

        # 使用BioPython选择距离在6埃范围内的原子
        pdb_parser = PDBParser(QUIET=True)        # 创建一个PDBParser对象，用于解析PDB文件。
        structure = pdb_parser.get_structure('Protein', pdb_file)    # 使用get_structure方法解析PDB文件，并将结构信息保存到名为structure的对象中。
        io = PDBIO()                    # 创建一个PDBIO对象，用于处理PDB文件的读取和保存。
        io.set_structure(structure)     # 将之前解析得到的结构信息设置为当前的PDB结构。
        io.save(f"{toFolder}/{protein_name}_cut10-{i}.pdb", DistanceSelect(reference_point, cutoff))   # 将距离在截止范围内的原子信息保存到一个新的PDB文件中。在保存时，会使用DistanceSelect类中定义的accept_atom方法来决定哪些原子被选中。

"""

'\nimport numpy as np\nfrom rdkit import Chem\nfrom rdkit.Chem import AllChem\nfrom Bio.PDB import PDBIO, PDBParser, Select\n\nclass DistanceSelect(Select):   # 定义一个名为DistanceSelect的类，继承自Bio.PDB的Select类。初始化时接收一个参考点和截止距离。\n    def __init__(self, reference_point, cutoff):\n        self.reference_point = reference_point\n        self.cutoff = cutoff\n\n    def accept_atom(self, atom):    # 在DistanceSelect类中定义一个名为accept_atom的方法。该方法计算输入原子与参考点的距离，如果距离小于等于截止距离，返回True，表示该原子被选中。\n        distance = np.linalg.norm(self.reference_point - np.array(atom.coord))\n        return distance <= self.cutoff\n\n\nfor protein_name in tqdm(ligand_segment_barycenter.keys()):\n    toFolder = f"./cut_off_protein_pocket/{protein_name}"\n    os.system(f"mkdir -p {toFolder}")\n\n    barycenter = ligand_segment_barycenter[protein_name]\n    for i in barycenter:\n        # 读取PDB文件\n        #pdb = "1a0q"\n        pdb_file = f"./pdbbind_files_copy/{protein_name}/{protein_name}_protein.pdb"\n\n        # 输入一个点的坐标\n     

In [26]:
# 多线程加速形式：

import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBIO, PDBParser, Select

from multiprocessing import Pool

class DistanceSelect(Select):   # 定义一个名为DistanceSelect的类，继承自Bio.PDB的Select类。初始化时接收一个参考点和截止距离。
    def __init__(self, reference_point, cutoff):
        self.reference_point = reference_point
        self.cutoff = cutoff

    def accept_atom(self, atom):    # 在DistanceSelect类中定义一个名为accept_atom的方法。该方法计算输入原子与参考点的距离，如果距离小于等于截止距离，返回True，表示该原子被选中。
        distance = np.linalg.norm(self.reference_point - np.array(atom.coord))
        return distance <= self.cutoff

def process_protein(protein_name):
    toFolder = f"./cut_off_protein_pocket_6A_test2/{protein_name}"

    if not os.path.exists(toFolder):
        os.system(f"mkdir -p {toFolder}")

    barycenter = ligand_segment_barycenter[protein_name]
    for i in barycenter:
        # 读取PDB文件
        #pdb = "1a0q"
        pdb_file = f"./pdbbind_files_copy/{protein_name}/{protein_name}_protein.pdb"

        # 输入一个点的坐标
        #reference_point = np.array([15.97316666666667, 21.566333333333333, 55.06100000000001])
        reference_point = np.array(barycenter[i])

        # 设置截止距离
        cutoff = 6.0  

        # 使用BioPython选择距离在6埃范围内的原子
        pdb_parser = PDBParser(QUIET=True)        # 创建一个PDBParser对象，用于解析PDB文件。
        structure = pdb_parser.get_structure('Protein', pdb_file)    # 使用get_structure方法解析PDB文件，并将结构信息保存到名为structure的对象中。
        io = PDBIO()                    # 创建一个PDBIO对象，用于处理PDB文件的读取和保存。
        io.set_structure(structure)     # 将之前解析得到的结构信息设置为当前的PDB结构。
        io.save(f"{toFolder}/{protein_name}_cut6-{i}.pdb", DistanceSelect(reference_point, cutoff))   # 将距离在截止范围内的原子信息保存到一个新的PDB文件中。在保存时，会使用DistanceSelect类中定义的accept_atom方法来决定哪些原子被选中。


protein_names = list(ligand_segment_barycenter.keys())

# 设置并行处理的进程数
num_processes = 4

"""
with Pool(num_processes) as p:
    p.map(process_protein, protein_names)     # 将process_protein函数应用到protein_names列表中的每个元素上
"""
with Pool(num_processes) as p:
    # 使用imap_unordered方法并在tqdm中设置total和desc参数i
    # 将imap_unordered的结果传递给tqdm，并设置total参数为protein_names列表的长度，以便tqdm知道总任务数量。desc参数设置了进度条的描述。
    for _ in tqdm(p.imap_unordered(process_protein, protein_names), total=len(protein_names), desc='Processing proteins'):   # 创建一个进程池并使用tqdm来显示处理进度。
        pass      

Processing proteins:  10%|▉         | 1838/19119 [10:20<1:37:11,  2.96it/s]


KeyboardInterrupt: 

In [ ]:
### 最后一次确认是用tankbind_new_rdkit python3.8来弄的！